In [ ]:
%load_ext autoreload
%autoreload 2

In [24]:
import pickle
import pvlib
import pandas as pd
from datetime import datetime

In [17]:
full_pv = pickle.load(open("./raw/generation.pickle", "rb"))

In [18]:
full_pv = pd.DataFrame({"Datetime":full_pv.times, "AC": full_pv.ac})

In [19]:
full_pv

,Datetime,AC
time,,
2006-01-01 00:10:00,2006-01-01 00:10:00,-3.03
2006-01-01 01:10:00,2006-01-01 01:10:00,-3.03
2006-01-01 02:10:00,2006-01-01 02:10:00,-3.03
2006-01-01 03:10:00,2006-01-01 03:10:00,-3.03
2006-01-01 04:10:00,2006-01-01 04:10:00,-3.03
...,...,...
2010-12-31 19:10:00,2010-12-31 19:10:00,-3.03
2010-12-31 20:10:00,2010-12-31 20:10:00,-3.03
2010-12-31 21:10:00,2010-12-31 21:10:00,-3.03


In [20]:
pv = full_pv[['Datetime', 'AC']].copy()
pv.loc[:,'Datetime'] = pd.to_datetime(pv.loc[:,'Datetime'])
pv.set_index('Datetime', inplace=True)
pv = pv.loc[:, ['AC']]
pv = pv.resample('min').interpolate(method='linear')

In [21]:
pv

,AC
Datetime,
2006-01-01 00:10:00,-3.03
2006-01-01 00:11:00,-3.03
2006-01-01 00:12:00,-3.03
2006-01-01 00:13:00,-3.03
2006-01-01 00:14:00,-3.03
...,...
2010-12-31 23:06:00,-3.03
2010-12-31 23:07:00,-3.03
2010-12-31 23:08:00,-3.03


In [22]:
pv_episodes = pv.groupby(pd.Grouper(freq='W'))

In [23]:
pv_episodes.describe()

AC                                                       \
              count      mean        std       min   25%   50%       75%   
Datetime                                                                   
2006-01-01   1430.0  1.730508  11.861138 -3.030000 -3.03 -3.03 -3.019739   
2006-01-08  10080.0  3.207000  14.176947 -3.030000 -3.03 -3.03  0.834513   
2006-01-15  10080.0  1.939853  11.700089 -3.030000 -3.03 -3.03 -0.934938   
2006-01-22  10080.0  6.654501  19.228682 -3.030000 -3.03 -3.03  4.989478   
2006-01-29  10080.0  5.393326  18.792913 -3.353673 -3.03 -3.03  3.804040   
...             ...       ...        ...       ...   ...   ...       ...   
2010-12-05  10080.0  6.404910  18.598541 -4.010676 -3.03 -3.03  1.588001   
2010-12-12  10080.0  3.315945  15.073696 -3.437605 -3.03 -3.03 -1.241439   
2010-12-19  10080.0  2.316614  12.192986 -3.030000 -3.03 -3.03 -2.104881   
2010-12-26  10080.0  4.858090  16.087377 -3.531632 -3.03 -3.03  0.618609   
2011-01-02   7151.0  4.938394  16.889054 -3.498425 -3.03 -3.03 -0.243066   

                       
                  max  
Datetime               
2006-01-01  51.639288  
2006-01-08  68.179388  
2006-01-15  65.029365  
2006-01-22  79.884108  
2006-01-29  94.812569  
...               ...  
2010-12-05  73.983002  
2010-12-12  69.827375  
2010-12-19  65.142728  
2010-12-26  63.218910  
2011-01-02  67.174872  

[262 rows x 8 columns]

In [25]:
pv_episodes = pd.concat([group.fillna(0) for name, group in pv_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2007, 1, 1) and group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))

In [26]:
pv_episodes.describe()

AC                                                       \
              count      mean        std       min   25%   50%       75%   
Datetime                                                                   
2007-01-07  10080.0  4.354325  16.213006 -3.345811 -3.03 -3.03 -1.240526   
2007-01-14  10080.0  5.274582  17.286283 -4.093479 -3.03 -3.03  1.830830   
2007-01-21  10080.0  5.476825  18.210548 -3.030000 -3.03 -3.03  0.109225   
2007-01-28  10080.0  8.311984  22.099629 -3.030000 -3.03 -3.03  6.446465   
2007-02-04  10080.0  6.041587  17.736726 -3.030000 -3.03 -3.03  9.364835   
...             ...       ...        ...       ...   ...   ...       ...   
2008-11-30  10080.0  6.744822  19.219951 -3.030000 -3.03 -3.03  5.067030   
2008-12-07  10080.0  3.824717  14.540284 -3.484874 -3.03 -3.03  1.861949   
2008-12-14  10080.0  4.796759  16.509024 -3.891251 -3.03 -3.03 -1.425037   
2008-12-21  10080.0  2.745637  11.871417 -3.952982 -3.03 -3.03  0.225522   
2008-12-28  10080.0  1.406385  11.634890 -3.030000 -3.03 -3.03 -2.024683   

                       
                  max  
Datetime               
2007-01-07  66.302596  
2007-01-14  67.927864  
2007-01-21  77.623347  
2007-01-28  90.799778  
2007-02-04  90.245043  
...               ...  
2008-11-30  77.291454  
2008-12-07  68.324537  
2008-12-14  66.912481  
2008-12-21  63.456489  
2008-12-28  62.449755  

[104 rows x 8 columns]

In [27]:
demand =  pd.HDFStore('./minutely/demand.h5')
with pd.HDFStore('./minutely/pv.h5') as store:
    # Save each DataFrame with a key
    i = 0
    for name, group in pv_episodes:
        if name.date() == demand[f"eps_{i}"].index[-1].date():
            store[f'eps_{i}'] = group
            i += 1
file = pd.HDFStore('./minutely/pv.h5')
print(len(file) == len(demand))
file.close()
demand.close()

True
